<a href="https://colab.research.google.com/github/mericamp97/-greenhouse-job-description-scraper-chrome-extension/blob/main/Bert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
file_path_profile = '/content/drive/My Drive/BT5153_Kaggle/profile.csv'
profile = pd.read_csv(file_path_profile)
file_path_trx = '/content/drive/My Drive/BT5153_Kaggle/trx_data.csv'
trx = pd.read_csv(file_path_trx)
file_path_trx = '/content/drive/My Drive/BT5153_Kaggle/train_label.csv'
train_label = pd.read_csv(file_path_trx)

#Feature Engineering

In [ ]:
trx['transaction_time'] = pd.to_datetime(trx['transaction_time'])

# Pre-calculate flags for each transaction type and asset type
trx['sell_flag'] = (trx['transaction_type'] == 'SELL').astype(int)
trx['buy_flag'] = (trx['transaction_type'] == 'BUY').astype(int)

# For asset types, you can use get_dummies for a one-hot encoding approach
asset_type_dummies = pd.get_dummies(trx['asset_type'])
trx = pd.concat([trx, asset_type_dummies], axis=1)

# Now group by 'user_id' and sum up the flags and dummies
agg_dict = {
    'sell_flag': 'sum',
    'buy_flag': 'sum',
    'crypto': 'sum',
    'gold': 'sum',
    'fx': 'sum',
    'gss': 'sum',
    'stock_index': 'sum',
    'idss': 'sum',
    'mfund': 'sum'
}

# Adding total transactions as count of rows
agg_dict['user_id'] = 'count'

trx_counts = trx.groupby('user_id').agg(agg_dict).rename(columns={'user_id': 'total_transactions'})

# Let's check the first few rows of the optimized trx_counts
print(trx_counts.head())


         sell_flag  buy_flag  crypto  gold  fx  gss  stock_index  idss  mfund  \
user_id                                                                         
0                0         3       2     1   0    0            0     0      0   
1                0         9       8     1   0    0            0     0      0   
2                1         0       0     1   0    0            0     0      0   
3                0         6       4     0   1    1            0     0      0   
4                2         1       0     2   0    0            0     0      1   

         total_transactions  
user_id                      
0                         3  
1                         9  
2                         1  
3                         6  
4                         3  


In [ ]:
trx[:10]

,transaction_time,user_id,transaction_type,asset_type,gtv,sell_flag,buy_flag,crypto,fx,gold,gss,idss,mfund,stock_index
0,2022-04-01 00:00:01.382,59345,SELL,crypto,571.296429,1,0,1,0,0,0,0,0,0
1,2022-04-01 00:00:02.218,76970,BUY,crypto,26.780357,0,1,1,0,0,0,0,0,0
2,2022-04-01 00:00:04.892,81533,BUY,crypto,8.916071,0,1,1,0,0,0,0,0,0
3,2022-04-01 00:00:05.002,2767,SELL,crypto,685.591071,1,0,1,0,0,0,0,0,0
4,2022-04-01 00:00:05.769,34438,SELL,crypto,81.142857,1,0,1,0,0,0,0,0,0
5,2022-04-01 00:00:07.239,79831,BUY,crypto,221.975000,0,1,1,0,0,0,0,0,0
6,2022-04-01 00:00:07.762,69419,SELL,crypto,244.057143,1,0,1,0,0,0,0,0,0
7,2022-04-01 00:00:08.210,74868,SELL,crypto,240.985714,1,0,1,0,0,0,0,0,0
8,2022-04-01 00:00:08.309,2880,BUY,crypto,362.433929,0,1,1,0,0,0,0,0,0
9,2022-04-01 00:00:08.724,39775,BUY,crypto,20.625000,0,1,1,0,0,0,0,0,0


In [ ]:

# Calculate GTV statistics
gtv_stats = trx.groupby('user_id')['gtv'].agg(['mean', 'median', 'max', 'min']).rename(columns={
    'mean': 'average_GTV',
    'median': 'median_GTV',
    'max': 'max_GTV',
    'min': 'min_GTV'
})

gtv_stats['max_minus_median_GTV'] = gtv_stats['max_GTV'] - gtv_stats['median_GTV']

# Display the first few rows of the modified GTV stats
print(gtv_stats.head())

         average_GTV  median_GTV      max_GTV     min_GTV  \
user_id                                                     
0          14.481548   16.669643    17.848214    8.926786   
1         270.889286   89.285714  1679.078571   89.282143   
2         162.785714  162.785714   162.785714  162.785714   
3         564.834226  535.711607   714.282143  442.687500   
4          19.985119   16.666071    26.750000   16.539286   

         max_minus_median_GTV  
user_id                        
0                    1.178571  
1                 1589.792857  
2                    0.000000  
3                  178.570536  
4                   10.083929  


In [ ]:
# Calculate average time between transactions in minutes
trx.sort_values(by=['user_id', 'transaction_time'], inplace=True)
trx['time_diff'] = trx.groupby('user_id')['transaction_time'].diff().dt.total_seconds() / 60
avg_time_diff = trx.groupby('user_id')['time_diff'].mean().to_frame('avg_time_between_transactions')

print(avg_time_diff[:5])

print(avg_time_diff['avg_time_between_transactions'].max())

         avg_time_between_transactions
user_id                               
0                            25.095825
1                          8375.102619
2                                  NaN
3                          8358.912597
4                          2659.806083
86359.67031666666


In [ ]:
# Calculate daily GTV percentage
trx['date'] = trx['transaction_time'].dt.date
daily_gtv = trx.groupby(['date', 'user_id'])['gtv'].sum().reset_index()
daily_total_gtv = trx.groupby('date')['gtv'].sum().reset_index(name='total_gtv')
daily_gtv = pd.merge(daily_gtv, daily_total_gtv, on='date')
daily_gtv['daily_gtv_percentage'] = daily_gtv['gtv'] / daily_gtv['total_gtv'] * 100
avg_daily_gtv_percentage = daily_gtv.groupby('user_id')['daily_gtv_percentage'].mean().to_frame('avg_daily_gtv_percentage')

print(avg_daily_gtv_percentage[:5])
print(avg_daily_gtv_percentage['avg_daily_gtv_percentage'].max())

         avg_daily_gtv_percentage
user_id                          
0                        0.000153
1                        0.003412
2                        0.000867
3                        0.003553
4                        0.000119
5.402543659339173


In [ ]:
#CALCULATE AND MERGE SUMS

# Group by user_id and asset_type, then sum the GTV
gtv_by_asset_user = trx.groupby(['user_id', 'asset_type'])['gtv'].sum()

# Unstack to create a DataFrame where each asset type is a column, and fill missing values with 0
gtv_by_asset_user = gtv_by_asset_user.unstack(fill_value=0)

# Rename the columns using a dictionary comprehension
# The keys of the dictionary are the current column names (i.e., asset types)
# and the values are the new column names in the format '{assetType}_sum'
gtv_by_asset_user.columns = [f'{col}_sum' for col in gtv_by_asset_user.columns]

# You can now merge this with your existing DataFrame 'profile' on 'user_id'
profile = profile.merge(gtv_by_asset_user, on='user_id', how='left', suffixes=('', '_total_gtv'))

In [ ]:
# Combine all the metrics
metrics = [trx_counts, gtv_stats, avg_time_diff, avg_daily_gtv_percentage]
for metric in metrics:
    profile = profile.merge(metric, on='user_id', how='left')


# Display the enriched profile DataFrame
print(profile.head())

   user_id mobile_brand_name mobile_marketing_name  age_in_year gender_name  \
0    53058              POCO             m3 pro 5g         45.0        Male   
1    74136             Nokia                     3         52.0        Male   
2   159548              POCO             m3 pro 5g         52.0      Female   
3   156814           OnePlus                     8         62.0        Male   
4   122581                LG              g7 thinq         48.0      Female   

  marital_status education_background income_level occupation   crypto_sum  \
0            NaN                  NaN          NaN        NaN  1117.446429   
1            NaN                  NaN          NaN        NaN     0.000000   
2            NaN                  NaN          NaN        NaN     0.000000   
3            NaN                  NaN          NaN        NaN     0.000000   
4            NaN                  NaN          NaN        NaN     0.000000   

   ...  idss  mfund  total_transactions  average_GTV   m

In [ ]:
import pandas as pd

# Assuming trx and profile are your DataFrames

# Filter for SELL transactions and sum GTV per user_id
total_gtv_sell = trx[trx['transaction_type'] == 'SELL'].groupby('user_id')['gtv'].sum().reset_index(name='total_gtv_sell')

# Filter for BUY transactions and sum GTV per user_id
total_gtv_buy = trx[trx['transaction_type'] == 'BUY'].groupby('user_id')['gtv'].sum().reset_index(name='total_gtv_buy')

# Merge the summaries back into the profile DataFrame
profile = profile.merge(total_gtv_sell, on='user_id', how='left')
profile = profile.merge(total_gtv_buy, on='user_id', how='left')

# Fill NaN values with 0 for users with no SELL or BUY transactions
profile['total_gtv_sell'].fillna(0, inplace=True)
profile['total_gtv_buy'].fillna(0, inplace=True)

# Display the updated profile DataFrame
print(profile.head())



   user_id mobile_brand_name mobile_marketing_name  age_in_year gender_name  \
0    53058              POCO             m3 pro 5g         45.0        Male   
1    74136             Nokia                     3         52.0        Male   
2   159548              POCO             m3 pro 5g         52.0      Female   
3   156814           OnePlus                     8         62.0        Male   
4   122581                LG              g7 thinq         48.0      Female   

  marital_status education_background income_level occupation   crypto_sum  \
0            NaN                  NaN          NaN        NaN  1117.446429   
1            NaN                  NaN          NaN        NaN     0.000000   
2            NaN                  NaN          NaN        NaN     0.000000   
3            NaN                  NaN          NaN        NaN     0.000000   
4            NaN                  NaN          NaN        NaN     0.000000   

   ...  total_transactions  average_GTV   median_GTV    

In [ ]:
profile.fillna({
    'total_transactions': 0, 'sell_flag': 0, 'buy_flag': 0, 'crypto': 0,
    'gold': 0, 'fx': 0, 'gss': 0, 'stock_index': 0,
    'idss': 0, 'mfund': 0,
}, inplace=True)

In [ ]:
profile.columns

Index(['user_id', 'mobile_brand_name', 'mobile_marketing_name', 'age_in_year',
       'gender_name', 'marital_status', 'education_background', 'income_level',
       'occupation', 'crypto_sum', 'fx_sum', 'gold_sum', 'gss_sum', 'idss_sum',
       'mfund_sum', 'stock_index_sum', 'sell_flag', 'buy_flag', 'crypto',
       'gold', 'fx', 'gss', 'stock_index', 'idss', 'mfund',
       'total_transactions', 'average_GTV', 'median_GTV', 'max_GTV', 'min_GTV',
       'max_minus_median_GTV', 'avg_time_between_transactions',
       'avg_daily_gtv_percentage', 'total_gtv_sell', 'total_gtv_buy'],
      dtype='object')

In [ ]:
text_df = profile.copy()

In [ ]:
import numpy as np
import pandas as pd

def create_conditional_text(df, col, sentence):
    """
    Creates a new column with a conditional sentence based on the presence of NaN in the specified column.

    Parameters:
    - df: DataFrame to modify.
    - col: Column to check for NaN.
    - sentence: Sentence pattern to use if the column value is not NaN.

    Returns:
    - Series with the constructed sentence or and empty string.
    """
    return df.apply(lambda row: sentence.format(row[col]) if pd.notna(row[col]) else "", axis=1)

# Define the sentence patterns for each column
sentence_patterns = [
    {'column': 'mobile_brand_name', 'text': "mobile brand name {}."},
    {'column': 'mobile_marketing_name', 'text': "mobile marketing campaign {}."},
    {'column': 'age_in_year', 'text': "{} years old."},
    {'column':'gender_name', 'text':"gender {} ."},
    {'column':'marital_status', 'text':"marital status {} ."},
    {'column':'education_background', 'text':"education background {}."},
    {'column':'income_level', 'text':"Income level {}."},
    {'column':'occupation', 'text':"occupation {}."},
    {'column':'crypto_sum', 'text':"{}$ crypto investments."},
    {'column':'crypto', 'text':"{} crypto transactions."},
    {'column':'fx_sum', 'text':"{}$ fx assets investment."},
    {'column':'fx', 'text':"{} fx assets transactions."},
    {'column':'gold_sum', 'text':"{}$ gold investment."},
    {'column':'gold', 'text':"{} gold transactions."},
    {'column':'gss_sum', 'text':"{}$ green, social or sustainable (GSS) assets investment."},
    {'column':'gss', 'text':"{} social or sustainable (GSS) assets transactions."},
    {'column':'idss_sum', 'text':"{}$ in Intraday Short Selling (IDSS) securities investment."},
    {'column':'idss', 'text':"{} Intraday Short Selling (IDSS) transactions."},
    {'column':'mfund_sum', 'text':"{}$ mutual funds investment."},
    {'column':'mfund', 'text':"{} mutual funds transactions."},
    {'column':'stock_index_sum', 'text':"{}$ in index stocks investment."},
    {'column':'stock_index', 'text':"{} index stock transactions."},
    {'column':'total_transactions', 'text':"{} total transactions."},
    {'column':'sell_flag', 'text':"{} SELL transactions."},
    {'column':'total_gtv_sell', 'text':"{}$ SELL transactions."},
    {'column':'buy_flag', 'text':"{} BUY transactions."},
    {'column':'total_gtv_buy', 'text':"{}$ BUY transactions."},
    {'column':'average_GTV', 'text':"{}$ average gross transaction value transactions."},
    {'column':'median_GTV', 'text':"{}$ median gross transaction value transactions."},
    {'column':'max_GTV', 'text':"{}$ highest gross transaction value."},
    {'column':'max_minus_median_GTV', 'text':"The difference in $ between my transaction with the highest gross transaction value and the median gross transaction value of all my transactions was {}$."},
    {'column':'avg_time_between_transactions', 'text':"On average, I do a transaction every {} minutes."}
    ]

# Loop through each pattern and apply the function
for pattern in sentence_patterns:
    col = pattern['column']
    text_col = 'text_' + col  # Construct the name for the new text column
    sentence = pattern['text']
    text_df[text_col] = create_conditional_text(text_df, col, sentence)


In [ ]:
text_df.columns

Index(['user_id', 'mobile_brand_name', 'mobile_marketing_name', 'age_in_year',
       'gender_name', 'marital_status', 'education_background', 'income_level',
       'occupation', 'crypto_sum', 'fx_sum', 'gold_sum', 'gss_sum', 'idss_sum',
       'mfund_sum', 'stock_index_sum', 'sell_flag', 'buy_flag', 'crypto',
       'gold', 'fx', 'gss', 'stock_index', 'idss', 'mfund',
       'total_transactions', 'average_GTV', 'median_GTV', 'max_GTV', 'min_GTV',
       'max_minus_median_GTV', 'avg_time_between_transactions',
       'avg_daily_gtv_percentage', 'total_gtv_sell', 'total_gtv_buy',
       'text_mobile_brand_name', 'text_mobile_marketing_name',
       'text_age_in_year', 'text_gender_name', 'text_marital_status',
       'text_education_background', 'text_income_level', 'text_occupation',
       'text_crypto_sum', 'text_crypto', 'text_fx_sum', 'text_fx',
       'text_gold_sum', 'text_gold', 'text_gss_sum', 'text_gss',
       'text_idss_sum', 'text_idss', 'text_mfund_sum', 'text_mfund',
   

In [ ]:
# Select columns that start with 'text'
text_columns = [col for col in text_df.columns if col.startswith('text')]

# Concatenate the values of these columns into a new column
text_df['user_description'] = text_df[text_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Display the DataFrame to check the new column
print(text_df[['user_id', 'user_description']].head())


   user_id                                   user_description
0    53058  mobile brand name POCO. mobile marketing campa...
1    74136  mobile brand name Nokia. mobile marketing camp...
2   159548  mobile brand name POCO. mobile marketing campa...
3   156814  mobile brand name OnePlus. mobile marketing ca...
4   122581  mobile brand name LG. mobile marketing campaig...


In [ ]:
print(text_df['user_description'].iloc[0])


mobile brand name POCO. mobile marketing campaign m3 pro 5g. 45.0 years old. gender Male .     1117.4464285714287$ crypto investments. 9 crypto transactions. 0.0$ fx assets investment. 0 fx assets transactions. 0.0$ gold investment. 0 gold transactions. 0.0$ green, social or sustainable (GSS) assets investment. 0 social or sustainable (GSS) assets transactions. 0.0$ in Intraday Short Selling (IDSS) securities investment. 0 Intraday Short Selling (IDSS) transactions. 0.0$ mutual funds investment. 0 mutual funds transactions. 0.0$ in index stocks investment. 0 index stock transactions. 9 total transactions. 4 SELL transactions. 487.5892857142858$ SELL transactions. 5 BUY transactions. 629.8571428571429$ BUY transactions. 124.16071428571429$ average gross transaction value transactions. 112.6267857142857$ median gross transaction value transactions. 171.75714285714287$ highest gross transaction value. The difference in $ between my transaction with the highest gross transaction value and 

In [ ]:
text_df = text_df.merge(train_label, on='user_id', how='left')

In [ ]:
df = text_df[['user_description','tgt']].copy()

df

,user_description,tgt
0,mobile brand name POCO. mobile marketing campa...,1.0
1,mobile brand name Nokia. mobile marketing camp...,1.0
2,mobile brand name POCO. mobile marketing campa...,0.0
3,mobile brand name OnePlus. mobile marketing ca...,1.0
4,mobile brand name LG. mobile marketing campaig...,0.0
...,...,...
188422,mobile brand name Samsung. mobile marketing ca...,1.0
188423,mobile brand name Samsung. mobile marketing ca...,1.0
188424,mobile brand name Samsung. mobile marketing ca...,0.0
188425,mobile brand name Samsung. mobile marketing ca...,0.0


# BERT

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
!pip install transformers

In [ ]:
!pip install cloud-tpu-client torch-xla


In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm


In [ ]:
from transformers import BertForSequenceClassification

# Load the pretrained BERT model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
)

# Acquire the current TPU core as device
device = xm.xla_device()

# Move the model to the TPU device
model = model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training Parameters

In [ ]:
# Larger batch sizes tend to be better, and we can fit this in memory.
batch_size = 32

# I used a smaller learning rate to combat over-fitting that I was seeing in the
# validation loss. I could probably try even smaller.
learning_rate = 1e-5

# Number of training epochs.
epochs = 4

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Initialize the maximum sentence length.
max_len = 0

# For every sentence...
for sent in df['user_description']:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length in `user_description`: ', max_len)



Loading BERT tokenizer...
Max sentence length in `user_description`:  362


In [ ]:
# Let's use a maximum length of 200.
max_len = 362

### Tokenize and Encode

In [ ]:
from transformers import BertTokenizer
import torch

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

print('Encoding all user descriptions in the dataset...')

# For every description...
for sent in df['user_description']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens=True,   # Add '[CLS]' and '[SEP]'
                        max_length=max_len,        # Pad & truncate all sentences.
                        truncation=True,
                        padding='max_length',
                        return_attention_mask=True,# Construct attn. masks.
                        return_tensors='pt',       # Return pytorch tensors.
                )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

print('DONE.')


Loading BERT tokenizer...
Encoding all user descriptions in the dataset...
DONE.


# Training

### Set Up

In [ ]:
from torch.utils.data import TensorDataset, random_split

# First, we'll separate the rows with non-null target values for training the model
# and the rows with null target values for later prediction.
mask = df['tgt'].notnull()

# Separate the dataframe into data for training and for prediction
df_train = df[mask]
df_predict = df[~mask]

# Convert the 'tgt' column in the training dataframe to a tensor
labels = torch.tensor(df_train['tgt'].values)

# Create the TensorDataset with input_ids, attention_masks, and labels for the training data
# Note: We need to filter the input_ids and attention_masks to only include the ones corresponding to the non-null 'tgt' values
dataset = TensorDataset(input_ids[mask], attention_masks[mask], labels)

# Define the size of the splits: train and validation
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Perform the split
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))

# We will save the indices of the rows with null 'tgt' for later use
predict_indices = df.index[~mask].tolist()

# The prediction dataset only includes the input_ids and attention_masks, without labels, for the rows with null 'tgt'
predict_dataset = TensorDataset(input_ids[~mask], attention_masks[~mask])

print('{:>5,} prediction samples'.format(len(predict_dataset)))
# Output the first few prediction indices as a check
predict_indices[:5]



135,666 training samples
15,075 validation samples
37,686 prediction samples


[7, 9, 11, 12, 13]

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoader for the training set. DataLoaders are used to shuffle the data and create batches.
train_dataloader = DataLoader(
    train_dataset,  # The TensorDataset created earlier for training
    sampler=RandomSampler(train_dataset),  # Sampling strategy is random to shuffle the data
    batch_size=batch_size  # The batch size
)

# Create the DataLoader for the validation set. Typically, you don't need to shuffle the validation data.
validation_dataloader = DataLoader(
    val_dataset,  # The TensorDataset created earlier for validation
    sampler=SequentialSampler(val_dataset),  # Sampling strategy is sequential as order doesn't matter for validation
    batch_size=batch_size  # The batch size
)



### Train

In [ ]:
from transformers import AdamW

# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = 1e-8
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples!)
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:

# Install PyTorch/XLA
!pip install cloud-tpu-client torch_xla

In [ ]:
import random
import numpy as np
import torch_xla.core.xla_model as xm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

for param in model.bert.parameters():
    param.requires_grad = False

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)

torch.manual_seed(seed_val)

#torch.cuda.manual_seed_all(seed_val)

# Imports
import torch_xla
import torch_xla.core.xla_model as xm

# Create a device object for TPU
device = xm.xla_device()


# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# Wrap your DataLoader with ParallelLoader
train_dataloader = pl.ParallelLoader(train_dataloader, [device]).per_device_loader(device)
validation_dataloader = pl.ParallelLoader(validation_dataloader, [device]).per_device_loader(device)

global_batch_size = batch_size

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    model.train()
    total_train_loss = 0

    # ParallelLoader creates a new DataLoader for each device
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        model.zero_grad()
        result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels, return_dict=True)

        loss = result.loss
        total_train_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Replace optimizer.step() with xm.optimizer_step(optimizer)
        # This ensures compatibility with TPUs.
        xm.optimizer_step(optimizer, barrier=True)  # Barrier ensures all cores wait for each other

        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # `xm.xla_device()` returns the current device (TPU core) this process is using.
        device = xm.xla_device()

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            result = model(b_input_ids,
                          token_type_ids=None,
                          attention_mask=b_input_mask,
                          labels=b_labels,
                          return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the validation loss. Use `xm.mesh_reduce` to aggregate loss values across all TPU cores.
        total_eval_loss += xm.mesh_reduce('total_eval_loss', loss.item(), lambda x: sum(x))

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += xm.mesh_reduce('total_eval_accuracy', flat_accuracy(logits, label_ids), lambda x: sum(x))


    # Calculate global average validation loss and accuracy.
    avg_val_loss = total_eval_loss / (len(validation_dataloader) * global_batch_size)
    avg_val_accuracy = total_eval_accuracy / (len(validation_dataloader) * global_batch_size)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# Results

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap (doesn't seem to work in Colab).
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
# Training Loss and Validation Loss
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

# Make Predictions to Test

In [ ]:
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
import torch_xla.distributed.parallel_loader as pl
import numpy as np
import pandas as pd

# Assuming predict_dataset is correctly defined as shown earlier
prediction_sampler = DistributedSampler(predict_dataset, num_replicas=xm.xrt_world_size(), rank=xm.get_ordinal(), shuffle=False)
prediction_dataloader = DataLoader(predict_dataset, sampler=prediction_sampler, batch_size=batch_size)

print(f'Predicting labels for {len(predict_dataset):,} test sentences...')

model.eval()
predictions = []

# Wrap your DataLoader with ParallelLoader for TPU compatibility
predict_dataloader = pl.ParallelLoader(prediction_dataloader, [device]).per_device_loader(device)

for batch in predict_dataloader:
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, return_dict=True)

    logits = result.logits
    logits = logits.detach().cpu().numpy()

    # Store predictions
    predictions.append(logits)

# Combine the results across all batches and select the index of the highest logit as the prediction
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

print('DONE.')


In [ ]:
# Ensure df_predict is in the same order as predictions
assert len(df_predict) == len(flat_predictions), "Mismatch between the number of predictions and the number of user IDs"

# Create a DataFrame with user_id and predictions
submission_df = pd.DataFrame({
    'user_id': df_predict.loc[predict_indices, 'user_id'].values,  # Assuming 'user_id' is a column in df_predict
    'pred_proba': flat_predictions
})

# Save to CSV file
submission_file_path = '/content/drive/My Drive/BT5153_Kaggle/predictions.csv'
submission_df.to_csv(submission_file_path, index=False)

print(f'Submission file saved to {submission_file_path}')


OTHER

import numpy as np
import pandas as pd

def create_conditional_text(df, col, sentence):
    """
    Creates a new column with a conditional sentence based on the presence of NaN in the specified column.
    
    Parameters:
    - df: DataFrame to modify.
    - col: Column to check for NaN.
    - sentence: Sentence pattern to use if the column value is not NaN.
    
    Returns:
    - Series with the constructed sentence or and empty string.
    """
    return df.apply(lambda row: sentence.format(row[col]) if pd.notna(row[col]) else "", axis=1)

# Define the sentence patterns for each column
sentence_patterns = [
    {'column': 'mobile_brand_name', 'text': "My mobile brand name is {}."},
    {'column': 'mobile_marketing_name', 'text': "I am targeted by a mobile marketing campaign called {}."},
    {'column': 'age_in_year', 'text': "I am {} years old."},
    {'column':'gender_name', 'text':"My gender is {} ."},
    {'column':'marital_status', 'text':"My marital status is {} ."},
    {'column':'education_background', 'text':"I was educated until  {} level."},
    {'column':'income_level', 'text':"I earn a yearly $ amout of {}."},
    {'column':'occupation', 'text':"My occupation is  {}."},
    {'column':'crypto_sum', 'text':"I invested a total of {}$ in crypto assets."},
    {'column':'crypto', 'text':"I did a total of {} crypto transactions."},
    {'column':'fx_sum', 'text':"I invested a total of {}$ in fx assets."},
    {'column':'fx', 'text':"I did a total of {} fx transactions."},
    {'column':'gold_sum', 'text':"I invested a total of {}$ in gold assets."},
    {'column':'gold', 'text':"I did a total of {} gold transactions."},
    {'column':'gss_sum', 'text':"I invested a total of {}$ in green, social or sustainable (GSS) assets."},
    {'column':'gss', 'text':"I did a total of {} social or sustainable (GSS) assets transactions."},
    {'column':'idss_sum', 'text':"I invested a total of {}$ in Intraday Short Selling (IDSS) securities."},
    {'column':'idss', 'text':"I did a total of {} Intraday Short Selling (IDSS) transactions."},
    {'column':'mfund_sum', 'text':"I invested a total of {}$ in mutual funds."},
    {'column':'mfund', 'text':"I did a total of {} mutual funds transactions."},
    {'column':'stock_index_sum', 'text':"I invested a total of {}$ in index stocks."},
    {'column':'stock_index', 'text':"I did a total of {} index stock transactions."},
    {'column':'total_transactions', 'text':"I did a total of {} transactions."},
    {'column':'sell_flag', 'text':"I did a total of {} SELL transactions."},
    {'column':'total_gtv_sell', 'text':"I did SELL transactions for a total value of {}$."},
    {'column':'buy_flag', 'text':"I did a total of {} BUY transactions."},
    {'column':'total_gtv_buy', 'text':"I did BUY transactions for a total value of {}$."},
    {'column':'average_GTV', 'text':"The average gross transaction value of my transactions was {}$."},
    {'column':'median_GTV', 'text':"The median gross transaction value of my transactions was {}$."},
    {'column':'max_GTV', 'text':"My transaction with the highest gross transaction value had a value of {}$."},
    {'column':'max_minus_median_GTV', 'text':"The difference in $ between my transaction with the highest gross transaction value and the median gross transaction value of all my transactions was {}$."},
    {'column':'avg_time_between_transactions', 'text':"On average, I do a transaction every {} minutes."}
    ]

# Loop through each pattern and apply the function
for pattern in sentence_patterns:
    col = pattern['column']
    text_col = 'text_' + col  # Construct the name for the new text column
    sentence = pattern['text']
    text_df[text_col] = create_conditional_text(text_df, col, sentence)
